In [1]:
from pathlib import Path

import catboost as cb
import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
data_path = Path("../data/")
seed = 42

In [3]:
def balanced_log_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    n0, n1 = np.bincount(y_true.astype(int))
    w0 = 1 / (n0 / len(y_true))
    w1 = 1 / (n1 / len(y_true))

    l0 = -w0 / n0 * np.sum(np.where(y_true == 0, 1, 0) * np.log(1 - y_pred))
    l1 = -w1 / n1 * np.sum(np.where(y_true != 0, 1, 0) * np.log(y_pred))

    return (l0 + l1) / (w0 + w1)


In [4]:
# read data
train_df = pd.read_csv(data_path / "train.csv")
test_df = pd.read_csv(data_path / "test.csv")
greeks_df = pd.read_csv(data_path / "greeks.csv")

In [5]:
# some columns have trailing spaces
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
feature_cols = train_df.columns.tolist()[1:-1]

In [6]:
# training
oof = np.zeros(len(train_df))
skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
test_probs = []

In [7]:
fold = 1
for train_idx, val_idx in skf.split(train_df, greeks_df.iloc[:, 1:-1]):

    print(f"Fold-{fold}".center(110, "-"))
    fold += 1

    X_train, y_train = (
        train_df.loc[train_idx, feature_cols],
        train_df.loc[train_idx, "Class"],
    )

    X_val, y_val = (
        train_df.loc[val_idx, feature_cols],
        train_df.loc[val_idx, "Class"],
    )

    params = {
        "iterations": 10000,
        "early_stopping_rounds": 1000,
        "use_best_model": True,
        "auto_class_weights": "Balanced",
        "grow_policy": "Lossguide",  # [SymmetricTree, Depthwise, Lossguide]
        # "l2_leaf_reg": 5, # [0, inf]
        # "random_strength": 2, # [1, inf)
        # "bagging_temperature": 1, #[0, inf)
        # "grow_policy": "Lossguide", #[SymmetricTree, Depthwise, Lossguide]
        # [Balanced, SqrtBalanced]
    }
    model = cb.CatBoostClassifier(**params)
    model.fit(
        X_train, y_train, eval_set=[(X_val, y_val)], cat_features=["EJ"], verbose=1000
    )

    preds = model.predict_proba(X_val)[:, 1]
    oof[val_idx] = preds

    # make test predictions
    test_preds = model.predict_proba(test_df.iloc[:, 1:])
    test_probs.append(test_preds)

print(f"OOF score: {balanced_log_loss(train_df['Class'], oof):.4f}".center(110, "-"))


----------------------------------------------------Fold-1----------------------------------------------------
Learning rate set to 0.009778
0:	learn: 0.6828959	test: 0.6845793	best: 0.6845793 (0)	total: 64.9ms	remaining: 10m 48s
1000:	learn: 0.0064560	test: 0.4702148	best: 0.4031063 (214)	total: 4.85s	remaining: 43.6s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.403106261
bestIteration = 214

Shrink model to first 215 iterations.
----------------------------------------------------Fold-2----------------------------------------------------
Learning rate set to 0.009778
0:	learn: 0.6827324	test: 0.6832329	best: 0.6832329 (0)	total: 3.64ms	remaining: 36.4s
1000:	learn: 0.0077930	test: 0.2777746	best: 0.2619686 (732)	total: 4.76s	remaining: 42.8s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.2619685527
bestIteration = 732

Shrink model to first 733 iterations.
----------------------------------------------------Fold-3-----------------------

In [8]:
# generate a submission file
test_probs = np.mean(test_probs, axis=0)
sub_df = pd.DataFrame(
    {"Id": test_df.Id, "Class_0": test_probs[:, 0], "Class_1": test_probs[:, 1]}
)
sub_df.to_csv("submission.csv", index=False)
